<a href="https://colab.research.google.com/github/Nekoiii/ML_Practices_colab/blob/main/Lora_test_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sd-scripts github: https://github.com/kohya-ss/sd-scripts

Reference article：
· https://tkstock.site/2023/05/19/googlecolab-stablediffusion-lora-finetuning/#i-7

· https://www.ipentec.com/document/software-stable-diffusion-lora-learn#section_12



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python --version

Python 3.10.12


In [3]:
!git clone https://github.com/kohya-ss/sd-scripts

Cloning into 'sd-scripts'...
remote: Enumerating objects: 3061, done.
remote: Counting objects: 100% (1474/1474), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 3061 (delta 1364), reused 1360 (delta 1317), pack-reused 1587
Receiving objects: 100% (3061/3061), 3.43 MiB | 29.04 MiB/s, done.
Resolving deltas: 100% (2119/2119), done.


In [4]:
%cd sd-scripts
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install  --upgrade -r requirements.txt
!pip install -U --pre triton
!pip install xformers==0.0.16rc425
%cd ../

/content/sd-scripts
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 73.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu118:
      Successfully uninstalled torchvision-0.15.2+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1+cu117 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 27.6 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 36.4 MB/s eta 0:00:00
/content


In [5]:
!accelerate config default --mixed_precision fp16

2023-06-12 00:36:00.763581: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 00:36:00.922484: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 00:36:00.960756: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-12 00:36:01.771998: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [6]:
%cd /content/sd-scripts
%cp -r /content/drive/MyDrive/datasets/imgs/resized_canbright_imgs_1_partial ./trainning_data

/content/sd-scripts


In [7]:
!python finetune/make_captions.py --batch_size 8 ./trainning_data
!python finetune/tag_images_by_wd14_tagger.py --batch_size 4 ./trainning_data

2023-06-12 00:36:17.225259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 00:36:17.388338: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 00:36:17.426937: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-12 00:36:18.246054: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [11]:
!python finetune/merge_captions_to_metadata.py ./trainning_data meta_cap.json
!python finetune/merge_dd_tags_to_metadata.py ./trainning_data --in_json meta_cap.json meta_cap_dd.json
!python finetune/clean_captions_and_tags.py meta_cap_dd.json meta_clean.json

2023-06-12 00:41:04.475133: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 00:41:04.644865: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 00:41:04.683288: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-12 00:41:05.508677: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [12]:
!python finetune/prepare_buckets_latents.py \
./trainning_data meta_clean.json meta_lat.json \
/content/drive/MyDrive/StableDifussion/models/anything-v4.5-pruned-fp16.ckpt \
--batch_size 4 \
--max_resolution 512,512 \
--mixed_precision no

2023-06-12 00:41:17.692641: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 00:41:17.858770: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 00:41:17.897613: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-12 00:41:18.734828: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [13]:
!pwd


/content/sd-scripts


In [14]:
!accelerate launch  \
--num_cpu_threads_per_process 1 \
train_network.py \
--pretrained_model_name_or_path=/content/drive/MyDrive/StableDifussion/models/anything-v4.5-pruned-fp16.ckpt \
--in_json meta_lat.json \
--train_data_dir=./trainning_data \
--output_dir=output_models \
--shuffle_caption \
--train_batch_size=1 \
--learning_rate=1e-4 \
--max_train_steps=400 \
--use_8bit_adam \
--xformers --gradient_checkpointing \
--mixed_precision=fp16 \
--save_every_n_epochs=10 \
--save_precision=fp16 \
--save_model_as=safetensors  \
--network_module=networks.lora

2023-06-12 00:57:31.121022: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 00:57:31.283390: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 00:57:31.321827: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-12 00:57:32.160787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [19]:
model_path='./output_models/last.safetensors'
save_path='/content/drive/MyDrive/StableDifussion/models/my_models/canbright_satoo/'
!mkdir -p '{save_path}'
!cp '{model_path}' '{save_path}'

In [ ]:
# write dataset config
import toml
instance_data_dir='/content/drive/MyDrive/datasets/imgs/with_txt_resized_canbright_imgs_1_partial/'

dataset_config={
    'general': {
        'enable_bucket': True
    },
    'datasets': [
        {
            'resolution': 512,
            'batch_size': 4,
            'subsets': [
                {
                    'image_dir': instance_data_dir,
                    'caption_extension': '.txt',
                    'num_repeats': 1
                }
            ]
        }
    ]
}
dataset_config_path='/content/drive/MyDrive/ML_Practices/Lora_test_1/lora/dataset_config.toml'
with open(dataset_config_path, 'w') as file:
    toml.dump(dataset_config, file)